# pYPKa_Z_ARF1 and pYPKa_E_ARF1

This Jupyter notebook describe the construction of E. coli vectors [pYPKa_Z_ARF1](pYPKa_Z_ARF1.gb) and [pYPKa_E_ARF1](pYPKa_E_ARF1.gb). 
These two vectors share backbone and insert, but in the former, the insert is cloned using the restriction
enzyme [ZraI](http://rebase.neb.com/rebase/enz/ZraI.html) while in the latter [EcoRV](http://rebase.neb.com/rebase/enz/EcoRV.html) is used.

The insert cloned in ZraI is meant to function as a promoter, while in the EcoRV site it is meant to be  used as a terminator.

Links to the sequence of each vector in Genbank format can be found at the bottom of this document.

![pYPKa_Z and pYPKa_E](figure_pYPKa_ZE.png "pYPKa_Z or pYPKa_E plasmid")

The Python package [pydna](https://pypi.python.org/pypi/pydna/) package is imported in the code cell below to provide 
the cloning functionality. There is a [publication](http://www.biomedcentral.com/1471-2105/16/142) describing pydna as well as
[documentation](http://pydna.readthedocs.io/) available online. Pydna is developed on [Github](https://github.com/BjornFJohansson/pydna).

In [1]:
from pydna.readers import read
from pydna.parsers import parse_primers
from pydna.genbank import Genbank
from pydna.amplify import pcr
from pydna.amplify import Anneal

The vector backbone pYPKa is read from a local [file](pYPKa.gb).

In [2]:
pYPKa = read("pYPKa.gb")

Both restriction enzymes are imported from [Biopython](http://biopython.org/wiki/Main_Page)

In [3]:
from Bio.Restriction import ZraI, EcoRV

The vector is cut with both enzymes.

In [4]:
pYPKa_ZraI  = pYPKa.linearize(ZraI)
pYPKa_EcoRV = pYPKa.linearize(EcoRV)

The template below comes from a Genbank [record](http://www.ncbi.nlm.nih.gov/nuccore/BK006938.2).
Access to Genbank is needed in order to download the template.
If you execute this script, change the email address below to your own.
Always tell Genbank who you are, when using their web service.

In [5]:
gb = Genbank("bjornjobb@gmail.com")

The template is downloaded from Genbank below.

In [6]:
template = gb.nucleotide("BK006938.2 REGION: 115800..116320")

The template is a 521 bp linear DNA fragment.

In [8]:
template

Gbank(BK006938.2  115800-116320)(-521)

The insert has the sequence shown below.

In [9]:
print(str(template.seq))

TACTTAAGAGCTAAACGAAAAGATAAGTATCTCGCTCAACCTTTTAATTTTGCACAGTTGAATTCTCGTCTGATACTTAAGAAAGTCATTGCCCACATATATAACAGTAGTAATAATGATAATGATAGCAATAGTTTTAAGCTAGCTACTTAGTTGCATTTTTCAATAGTTTAGTAAAAAAAGTCACGCAATAAGCTCTCAAGAAGCCACTAATACCGTAATGATAGCAGTTTATTGTAGAAAAACCATGTTATTACCCTTCCCTTTTTATTTCTTTTCGCGTTGCAAATCACATATAACGAGGTGGCTTGTATTTGTCAAACCAAAAAAAAAAATGAAAATCGAAAAATGGAAAAACAGAGAGAGAAACGGAATCTTTGACACGTTCTCGATCCACTTGTTTATCGAGGTGGTTTTTATAAGTCTTACTAGATAGAAAGTTCATTTTGTTTTGAAACTTTTTGGAACCTCTGGCATTGAAGGTATAAGAAAGAACTCAAACAGGTTTAATAGAATTAAAA


The seguid checksum of the template should be

```I3W5nJJqK8ygBAdTH0gbQgTpGqw```

In [10]:
template.seguid()

I3W5nJJqK8ygBAdTH0gbQgTpGqw

Two primers are used to amplify the insert:

In [11]:
fp,rp = parse_primers(""">1006_fw521
                         ttaaatTACTTAAGAGCTAAACGAAAA
                         >1005_rv521
                         taattaaTTTTAATTCTATTAAACCTGTT""")

PCR to create the insert using the primers above.

In [12]:
prd = pcr(fp, rp, template)
assert str(fp.seq) in prd

The PCR product has this length in bp.

In [13]:
len(prd)

534

A figure of the primers annealing on template.

In [14]:
prd.figure()

      5TACTTAAGAGCTAAACGAAAA...AACAGGTTTAATAGAATTAAAA3
                               |||||||||||||||||||||| tm 47.0 (dbd) 52.7
                              3TTGTCCAAATTATCTTAATTTTaattaat5
5ttaaatTACTTAAGAGCTAAACGAAAA3
       ||||||||||||||||||||| tm 50.5 (dbd) 54.7
      3ATGAATTCTCGATTTGCTTTT...TTGTCCAAATTATCTTAATTTT5

A suggested PCR program.

In [15]:
prd.program()


Taq (rate 30 nt/s) 35 cycles             |534bp
95.0°C    |95.0°C                 |      |Tm formula: Biopython Tm_NN
|_________|_____          72.0°C  |72.0°C|SaltC 50mM
| 03min00s|30s  \         ________|______|Primer1C 1.0µM
|         |      \ 49.2°C/ 0min16s| 5min |Primer2C 1.0µM
|         |       \_____/         |      |GC 31%
|         |         30s           |      |4-12°C

The final vectors are:

In [16]:
pYPKa_Z_ARF1 = (pYPKa_ZraI  + prd).looped().synced(pYPKa)
pYPKa_E_ARF1 = (pYPKa_EcoRV + prd).looped().synced(pYPKa)

The final vectors with reverse inserts are created below. These vectors theoretically make up
fifty percent of the clones. The PCR strategy outlined below can be used to identify clones with the insert
in the correct orientation.

In [17]:
pYPKa_Z_ARF1b = (pYPKa_ZraI  + prd.rc()).looped().synced(pYPKa)
pYPKa_E_ARF1b = (pYPKa_EcoRV + prd.rc()).looped().synced(pYPKa)

A combination of yeast pathway kit standard primers and the primers above are
used for the strategy to identify correct clones.
Standard primers used in the yeast are listed [here](standard_primers.txt).

In [18]:
p = { x.id: x for x in parse_primers(""">577
                                         gttctgatcctcgagcatcttaagaattc
                                         >578
                                         gttcttgtctcattgccacattcataagt
                                         >468
                                         gtcgaggaacgccaggttgcccact
                                         >467
                                         ATTTAAatcctgatgcgtttgtctgcacaga
                                         >567
                                         GTcggctgcaggtcactagtgag
                                         >568
                                         GTGCcatctgtgcagacaaacg
                                         >775
                                         gcggccgctgacTTAAAT
                                         >778
                                         ggtaaatccggatTAATTAA
                                         >342
                                         CCTTTTTACGGTTCCTGGCCT""") }

## Diagnostic PCR confirmation

The correct structure of pYPKa_Z_ARF1 is confirmed by PCR using standard primers
577 and 342 that are vector specific together with the ARF1fw primer specific for the insert
in a multiplex PCR reaction with
all three primers present.

Two PCR products are expected if the insert was cloned, the sizes depend
on the orientation. If the vector is empty or contains another insert, only one
product is formed.

#### Expected PCR products sizes from pYPKa_Z_ARF1:

pYPKa_Z_ARF1 with insert in correct orientation.

In [19]:
Anneal( (p['577'], p['342'], fp), pYPKa_Z_ARF1).products

[Amplicon(1468), Amplicon(1300)]

pYPKa_Z_ARF1 with insert in reverse orientation.

In [20]:
Anneal( (p['577'], p['342'], fp), pYPKa_Z_ARF1b).products

[Amplicon(1468), Amplicon(702)]

Empty pYPKa clone.

In [21]:
Anneal( (p['577'], p['342'], fp), pYPKa).products

[Amplicon(934)]

#### Expected PCR products sizes pYPKa_E_ARF1:

pYPKa_E_ARF1 with insert in correct orientation.

In [22]:
Anneal( (p['577'], p['342'], fp), pYPKa_E_ARF1).products

[Amplicon(1468), Amplicon(1219)]

pYPKa_E_ARF1 with insert in reverse orientation.

In [23]:
Anneal( (p['577'], p['342'], fp), pYPKa_E_ARF1b).products


[Amplicon(1468), Amplicon(783)]

Calculate cseguid checksums for the resulting plasmids for future reference.
This checksum that uniquely describes a circular double stranded
sequence. See this [blog post](https://ochsavidare.blogspot.com/2016/02/checksum-for-circular-biological.html) for more info.

pYPKa_Z_ARF1.cseguid() should be btkuHSLWXbNXXY49C107ALQiCWE

In [24]:
print(pYPKa_Z_ARF1.cseguid())
assert pYPKa_Z_ARF1.cseguid() == "btkuHSLWXbNXXY49C107ALQiCWE"

btkuHSLWXbNXXY49C107ALQiCWE


pYPKa_E_ARF1.cseguid() should be 4qvEQ4qanpPOrcuO3LsGtYHxIlk

In [25]:
print(pYPKa_E_ARF1.cseguid())
assert pYPKa_E_ARF1.cseguid() == "4qvEQ4qanpPOrcuO3LsGtYHxIlk"

4qvEQ4qanpPOrcuO3LsGtYHxIlk


The sequences are named based on the name of the cloned insert.

In [26]:
pYPKa_Z_ARF1.locus = "pYPKa_Z_ARF1"[:16]
pYPKa_E_ARF1.locus = "pYPKa_Z_ARF1"[:16]

Stamp sequence with cseguid checksum. This can be used to verify the
integrity of the sequence file.

In [27]:
pYPKa_Z_ARF1.stamp()
pYPKa_E_ARF1.stamp()

cSEGUID_4qvEQ4qanpPOrcuO3LsGtYHxIlk

pYPKa_Z_ARF1 is written to a local file:

In [28]:
pYPKa_Z_ARF1.write("pYPKa_Z_ARF1.gb")

pYPKa_Z_ARF1.gb

pYPKa_E_ARF1 is written to a local file:

In [29]:
pYPKa_E_ARF1.write("pYPKa_E_ARF1.gb")

pYPKa_E_ARF1.gb